In [2]:
import os
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext


In [3]:
conf = SparkConf().setAppName('G101HW1').setMaster("local[*]")
sc = SparkContext(conf=conf)

22/04/04 23:31:35 WARN Utils: Your hostname, eunoia resolves to a loopback address: 127.0.1.1; using 192.168.127.57 instead (on interface wlp1s0)
22/04/04 23:31:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 23:31:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
""" 
    K = <int> Number of Partitions
    H = <int> Number of Products with Highest Popularity
    S = <str> Name of the Country -- 'all' Means all Countries
    dataset_path = Path of the Dataset File

"""
K=4
H=5
S='all'
dataset_path = 'sample_10000.csv'
# sqlContext = SQLContext(sc)
rawData = sc.textFile(dataset_path,K).cache()
rawData.repartition(K)
rawData.collect()

['536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United_Kingdom',
 '536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United_Kingdom',
 '536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,2010-12-01 08:34:00,3.75,13047,United_Kingdom',
 '536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047,United_Kingdom',
 '536369,21756,BATH BUILDING BLOCK WORD,3,2010-12-01 08:35:00,5.95,13047,United_Kingdom',
 '536370,22659,LUNCH BOX I LOVE LONDON,24,2010-12-01 08:45:00,1.95,12583,France',
 "536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS ,80,2010-12-01 09:00:00,2.55,13748,United_Kingdom",
 '536372,22633,HAND WARMER UNION JACK,6,2010-12-01 09:01:00,1.85,17850,United_Kingdom',
 '536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-12-01 09:09:00,10.95,15100,United_Kingdom',
 '536375,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:32:00,2.75,17850,United_Kingdom',
 '536376,21733,RED HANGING HEART T-LIGHT HOLDER,64,2010-12-01 09:32:00,2.5

In [7]:
rawData.count()

10000

In [8]:
rawData.getNumPartitions()

4

In [83]:
def productCustomer(row, country='all'):
    """
    row = [0:TransactionID, 1:ProductID, 2:Description, 3:Quantity, 4:InvoiceDate, 5:UnitPrice, 6:CustomerID, 7:Country]
    """
    s = row.split(',')
    if int(s[3]) > 0:
        if country == 'all':
            return ((s[1], int(s[6])),0)
        elif s[7] == country:
            return ((s[1], int(s[6])),0)

In [89]:
product_customer = rawData.map(lambda row: productCustomer(row,S) if productCustomer(row,S) else pass)

SyntaxError: invalid syntax (3985924881.py, line 1)

In [85]:
product_customer.count()

[(('84029E', 17850), 0),
 (('22632', 17850), 0),
 (('22749', 13047), 0),
 (('22960', 13047), 0),
 (('21756', 13047), 0),
 (('22659', 12583), 0),
 (('22086', 13748), 0),
 (('22633', 17850), 0),
 (('21258', 15100), 0),
 (('84406B', 17850), 0),
 (('21733', 15291), 0),
 (('22632', 17850), 0),
 (('21559', 14688), 0),
 (('22961', 17809), 0),
 (('15056N', 15311), 0),
 (('22798', 16098), 0),
 (('22961', 17420), 0),
 (('85099C', 16029), 0),
 (('22464', 16250), 0),
 (('22196', 12431), 0),
 (('85123A', 17511), 0),
 (('22150', 13705), 0),
 (('22180', 13747), 0),
 (('21484', 13767), 0),
 (('71053', 17850), 0),
 (('22114', 13448), 0),
 (('22632', 17850), 0),
 (('22969', 13448), 0),
 (('22837', 15513), 0),
 (('POST', 12791), 0),
 (('22383', 16218), 0),
 (('20914', 14045), 0),
 (('22752', 17850), 0),
 (('22633', 17850), 0),
 (('48194', 14307), 0),
 (('22111', 17908), 0),
 (('22077', 17920), 0),
 (('22837', 18085), 0),
 (('22637', 13758), 0),
 (('84879', 14849), 0),
 (('82552', 14849), 0),
 (('22960', 

In [86]:
p = product_customer.groupByKey().map(lambda x: x[0])
p.collect()

[('22961', 17809),
 ('22464', 16250),
 ('21484', 13767),
 ('22114', 13448),
 ('POST', 12791),
 ('22960', 14849),
 ('84347', 17897),
 ('20974', 14078),
 ('21258', 14237),
 ('22834', 15350),
 ('21915', 14911),
 ('22960', 13093),
 ('72741', 13468),
 ('22646', 13777),
 ('22837', 17690),
 ('22730', 18229),
 ('22969', 13065),
 ('21733', 17732),
 ('22865', 15752),
 ('21137', 17925),
 ('22961', 17017),
 ('22776', 15658),
 ('22086', 15070),
 ('22469', 15260),
 ('22909', 14060),
 ('20679', 17850),
 ('21232', 13491),
 ('22661', 17581),
 ('22865', 17460),
 ('22969', 14829),
 ('84029E', 17659),
 ('85014A', 13011),
 ('22910', 18239),
 ('20726', 17552),
 ('21231', 17135),
 ('84347', 16905),
 ('48138', 14589),
 ('84692', 13013),
 ('22943', 17809),
 ('22890', 14210),
 ('47590A', 16477),
 ('22632', 17819),
 ('21479', 17238),
 ('37446', 13081),
 ('21034', 17243),
 ('21754', 15465),
 ('79321', 16353),
 ('15036', 12748),
 ('21485', 15570),
 ('22193', 15808),
 ('21754', 14161),
 ('22867', 13767),
 ('22843',

In [87]:
product_customer = (rawData.map(lambda row: productCustomer(row, S)).filter(lambda row: row)
                    .groupByKey()
                    .map(lambda x : x[0]))

In [88]:
product_customer.count()

9104